In [1]:
def explore_scaling_relationships(min_n, max_n, m, max_bit_length):

    n_vals = list(range(min_n, max_n + 1))
    results = []

    #print(f"\n Starting bit_length {bit_len}")
    for bit_len in range(3, max_bit_length + 1):  
        for num_ones in range(1, bit_len + 1):
            binary_str = generate_binary_string(bit_len, num_ones)
            commutator_norms = []

            for n in n_vals:
                A = laplace(n)
                B = potential(n, m)
                comm = nested_commutator(binary_str, A, B)
                commutator_norms.append(np.linalg.norm(comm, 2))

            threshold = 1e-14

            # Step 2: Print norms to debug
            print(f"\nBitstring: {binary_str} | Norms:", [f"{val:.2e}" for val in commutator_norms])
            
            valid_data = [(n, norm) for n, norm in zip(n_vals, commutator_norms) if np.isfinite(norm) and norm > threshold]

            
            if len(valid_data) >= 2:
                # Log-log regression: log(norm) = k * log(n) + log(C)
                logs_n = np.log([v[0] for v in valid_data])
                logs_comm = np.log([v[1] for v in valid_data])
                k_fit, log_C = np.polyfit(logs_n, logs_comm, 1)
                C_fit = np.exp(log_C)
                zero_flag = False
            else:
                # Skip fit if too few valid points
                k_fit = None
                C_fit = None
                zero_flag = True
                print(f" Skipping fit: only {len(valid_data)} valid norms > {threshold}")

            

          
            results.append({
                "bit_length": bit_len,
                "num_ones": num_ones,
                "expected_k": num_ones,
                "fitted_k": k_fit,
                "C": C_fit,
                "binary_str": binary_str,
                "zero_commutator": zero_flag
            })

    df = pd.DataFrame(results)
    display(df)

    print(f"\n Total rows collected: {len(results)}")
    return df

In [ ]:
df = explore_scaling_relationships(
    min_n=4,
    max_n=200,
    m=100,
    max_bit_length=10
)